<h3>Глобальные переменные</h3>

In [1]:
transformer_model_path = 'sagteam/xlm-roberta-large-sag'#'sagteam/xlm-roberta-large-sag'
USE_CUDA = True
ADD_CONTEXT = False

<h3>Векторизуем словарь Meddra</h3>

In [2]:
from vectorization import ConceptVectorizer

In [3]:
#альтернатива
CV = ConceptVectorizer(transformer_model_path, '../../Data/External/pt_rus.asc', \
                             use_concept_less=False, use_model=False, use_cuda=USE_CUDA)
CV_with_conceptless = ConceptVectorizer(transformer_model_path, '../../Data/External/pt_rus.asc', \
                             use_concept_less=True, use_model=False, use_cuda=USE_CUDA)

Расчет вложений словаря

In [4]:
#используется только для векторизации словаря
#когда вектора готовы, проще их загрузить ячейкой с загрузкой
CV_train.fit_transform(mode='mean_pooling')

getting concept embeddings in mean_pooling mode...
moving to cuda... device name cuda:0
Compute embeddings...


100%|██████████| 25077/25077 [08:21<00:00, 50.02it/s]


Embedding aggregation...
Concept embeddings have computed in 501.371141910553 seconds


Сохранение рассчитанных векторов концептов

In [5]:
import torch

torch.save(CV_train.thesaurus_embeddings, './Model_weights/xlmrobertasag_thesaurus_embeddings.pt')

Загрузка готовых вложений (если они есть). Загружать вложения после fit_transform() не надо

In [4]:
import torch

CV.thesaurus_embeddings = torch.load('./Model_weights/xlmrobertasag_thesaurus_embeddings.pt')
CV.normalization_mode = 'mean_pooling'

In [5]:
CV.thesaurus_embeddings.size()

torch.Size([25077, 1024])

<h3>Обработка датасета RDRS</h3>

In [6]:
from tools.parse_RDRS import *
from sklearn.model_selection import KFold

In [7]:
RDRS_path = '../../Data/Raw/medNorm_16022022.jsonlines'
ds = np.array(parse_RDRS(RDRS_path))

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)
folds = []

all_splits = {}

id_folds=1
for train_index, test_index in kf.split(ds):
    X_test = ds[test_index]
    X_train = ds[train_index]
    X_train, X_val = train_test_split(X_train, test_size=0.15, random_state=42)

    ds_parsed, errors = parse_json_ds(X_train=X_train, \
                                 X_val=X_val, \
                                 X_test=X_test, \
                                 CV=CV, \
                                 CV_with_conceptless = CV_with_conceptless, \
                                 add_context=False)
    
    train_concepts, test_concepts = ds_parsed['train']['concepts'], ds_parsed['test']['concepts']
    train_phrases, test_phrases = ds_parsed['train']['phrases'], ds_parsed['test']['phrases']
    
    all_splits[id_folds] = ds_parsed
    
    #Посмотрим на статистику разбиения
    print('Сплит %s'%id_folds)
    print('Всего фраз в трейне: %s'%len(train_phrases))
    print('Всего фраз в тесте: %s'%len(test_phrases))

    print('Уникальных фраз в трейне: %s'%len(set(train_phrases)))
    print('Уникальных фраз в тесте: %s'%len(set(test_phrases)))
    
    print('%s концептов не входящих либо в трейн, либо в тест'%len(set.union(set(train_concepts), set(test_concepts)) - set.intersection(set(test_concepts), set(train_concepts))))
    print('%s концептов, которые есть в тесте, но нет в трейне'%len(set(test_concepts) - set(train_concepts)))
    print('%s концептов, которые есть в трейне, но нет в тесте'%len(set(train_concepts) - set(test_concepts)))
    print('\n')
    id_folds+=1

Сплит 1
Всего фраз в трейне: 8194
Всего фраз в тесте: 2211
Уникальных фраз в трейне: 4452
Уникальных фраз в тесте: 1390
568 концептов не входящих либо в трейн, либо в тест
96 концептов, которые есть в тесте, но нет в трейне
472 концептов, которые есть в трейне, но нет в тесте


Сплит 2
Всего фраз в трейне: 8021
Всего фраз в тесте: 2442
Уникальных фраз в трейне: 4371
Уникальных фраз в тесте: 1544
546 концептов не входящих либо в трейн, либо в тест
100 концептов, которые есть в тесте, но нет в трейне
446 концептов, которые есть в трейне, но нет в тесте


Сплит 3
Всего фраз в трейне: 8134
Всего фраз в тесте: 2402
Уникальных фраз в трейне: 4444
Уникальных фраз в тесте: 1488
578 концептов не входящих либо в трейн, либо в тест
93 концептов, которые есть в тесте, но нет в трейне
485 концептов, которые есть в трейне, но нет в тесте


Сплит 4
Всего фраз в трейне: 8116
Всего фраз в тесте: 2384
Уникальных фраз в трейне: 4385
Уникальных фраз в тесте: 1541
542 концептов не входящих либо в трейн, ли

<h2>Roberta sag кроссвалидация с контекстом</h2>

In [10]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np
from tools.pytorchtools import EarlyStopping
import pandas as pd
from torch.optim.lr_scheduler import CyclicLR
from dataset import *
from models import CADEC_SoTa

#Гиперпараметры
batch_size=16
epochs = 50

all_splits_score_dfs = []

input_format = 'sentences' if ADD_CONTEXT else 'phrases'

for fold_num in range(1, n_splits+1):
    print('Сплит %s'%fold_num)
    torch.manual_seed(0)
    np.random.seed(0)
    best_model_path = './Model_weights/robertasag_cyclic_lr_es_split_%s.pt'%fold_num
    
    #фразы могут быть с контекстом или без контекста, в зависимости от флага ADD_CONTEXT
    train_phrases, train_concepts = all_splits[fold_num]['train'][input_format], all_splits[fold_num]['train']['concepts']
    val_phrases, val_concepts = all_splits[fold_num]['validation'][input_format], all_splits[fold_num]['validation']['concepts']
    test_phrases, test_concepts = all_splits[fold_num]['test'][input_format], all_splits[fold_num]['test']['concepts']
    
    if not ADD_CONTEXT:
        ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
        ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)
        ds_val = MedNormDataset(val_phrases, val_concepts, CV, use_cuda=USE_CUDA)
    else:
        ds_train = MedNormContextDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
        ds_test = MedNormContextDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)
        ds_val = MedNormContextDataset(val_phrases, val_concepts, CV, use_cuda=USE_CUDA)
        
    net = CADEC_SoTa(transformer_model_path, CV.thesaurus_embeddings)
    #перемещение модели на вычислительные мощности, либо cpu, либо cuda
    device = 'cuda' if USE_CUDA else 'cpu'
    net.to(device)
    #net = torch.load('./Model_weights/rubert_only_additional_corp_v2_30ep.pt')
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.00003) #, cycle_momentum=True
    scheduler = CyclicLR(optimizer, base_lr=0.00004, max_lr=0.0002, cycle_momentum=False)
    scaler = torch.cuda.amp.GradScaler()
    early_stopping = EarlyStopping(path=best_model_path, mode='max', verbose=True)

    trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                              shuffle=False, num_workers=0)
    testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)
    valloader = torch.utils.data.DataLoader(ds_val, batch_size=1, shuffle=False)

    valid_evaluator = Evaluator(train_concepts, val_concepts)
    test_evaluator = Evaluator(train_concepts, test_concepts)

    initial_loss = None

    for epoch in range(1, epochs+1):
        #обучение модели в эпохе
        net.train()
        with tqdm(trainloader, unit="batch") as tepoch:
            for data in tepoch:

                tepoch.set_description(f"Epoch {epoch}")

                inputs = data['tokenized_phrases']
                labels = data['one_hot_labels']

                optimizer.zero_grad()
                if USE_CUDA:
                    with torch.cuda.amp.autocast():
                        outputs = net(inputs)['output']
                        loss = criterion(outputs, labels)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scheduler.step()
                    scaler.update()
                else:
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                if initial_loss is None:
                    initial_loss = loss.item()
                tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
        #расчет точностей модели после эпохи
        net.eval()
        model_answers=[]
        with tqdm(valloader, unit="batch") as eval_process:
            for data in eval_process:

                inputs = data['tokenized_phrases']

                with torch.no_grad():
                    outputs_dict = net(inputs)
                    pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

                model_answers.append(pred_meddra_code)

        valid_evaluator.get_all_f1_scores(model_answers)
        early_stopping(valid_evaluator.f1_scores_df['All']['f1 macro']['score'], net)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    net.load_state_dict(torch.load(best_model_path))



    print('Evaluating on test dataset')
    net.eval()
    model_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

            model_answers.append(pred_meddra_code)

        test_evaluator.get_all_f1_scores(model_answers)
    all_splits_score_dfs.append(test_evaluator.f1_scores_df)

columns_tuples = [('All', 'f1 micro'), 
                 ('All', 'f1 macro'),

                 ('In VOC', 'f1 micro'), 
                 ('In VOC', 'f1 macro'),

                 ('Out of VOC', 'f1 micro'), 
                 ('Out of VOC', 'f1 macro')]
col_index = pd.MultiIndex.from_tuples(columns_tuples)
f1_mean_scores_df = pd.DataFrame(columns=col_index, index=['mean score', 'std'])
f1_mean_scores_df.loc['std', :] = np.std(np.array(all_splits_score_dfs), axis = 0)
f1_mean_scores_df.loc['mean score', :] = np.mean(np.array(all_splits_score_dfs), axis = 0)
f1_mean_scores_df

Сплит 1


Some weights of the model checkpoint at sagteam/xlm-roberta-large-sag were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at sagteam/xlm-roberta-large-sag and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-

All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.198935  0.043445  0.209711  0.053592   0.012195  0.011628

Validation metric increased (-inf --> 0.043445).  Saving model ...


100%|██████████| 1503/1503 [00:33<00:00, 44.56batch/s]


...

100%|██████████| 2402/2402 [00:49<00:00, 48.10batch/s]


All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.716486  0.276004  0.753968  0.348254    0.08209  0.035714

Сплит 4


Some weights of the model checkpoint at sagteam/xlm-roberta-large-sag were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at sagteam/xlm-roberta-large-sag and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 31.75 GiB total capacity; 29.94 GiB already allocated; 15.69 MiB free; 30.37 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

<h2>Эксперимент с RuBERT</h2>

Подгружается rubert, который 30 эпох учился на корпусе<br> 
с переведенными с английского фразами из англоязычных корпусов и их pt концептами<br>
а также содержащий фразы из llt и соответствующие им концепты<br>
всего в этом корпусе было порядка ~90 тыс. примеров<br>

In [7]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np
from tools.pytorchtools import EarlyStopping
import pandas as pd
from torch.optim.lr_scheduler import CyclicLR
from dataset import MedNormDataset


#Гиперпараметры
batch_size=16
epochs = 50

all_splits_score_dfs = []

for fold_num in range(1, n_splits+1):
    print('Сплит %s'%fold_num)
    best_model_path = './Model_weights/rubert_cyclic_lr_es_split_%s.pt'%fold_num
    
    train_phrases, train_concepts = all_splits[fold_num]['train']['phrases'], all_splits[fold_num]['train']['concepts']
    val_phrases, val_concepts = all_splits[fold_num]['validation']['phrases'], all_splits[fold_num]['validation']['concepts']
    test_phrases, test_concepts = all_splits[fold_num]['test']['phrases'], all_splits[fold_num]['test']['concepts']
    
    ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
    ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)
    ds_val = MedNormDataset(val_phrases, val_concepts, CV, use_cuda=USE_CUDA)

    net = torch.load('./Model_weights/rubert_only_additional_corp_v2_30ep.pt')

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0001) #, cycle_momentum=True
    scheduler = CyclicLR(optimizer, base_lr=0.00002, max_lr=0.0001, cycle_momentum=False)
    scaler = torch.cuda.amp.GradScaler()
    early_stopping = EarlyStopping(path=best_model_path, mode='max', verbose=True)

    trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                              shuffle=False, num_workers=0)
    testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)
    valloader = torch.utils.data.DataLoader(ds_val, batch_size=1, shuffle=False)

    valid_evaluator = Evaluator(train_concepts, val_concepts)
    test_evaluator = Evaluator(train_concepts, test_concepts)

    initial_loss = None

    for epoch in range(1, epochs+1):
        #обучение модели в эпохе
        net.train()
        with tqdm(trainloader, unit="batch") as tepoch:
            for data in tepoch:

                tepoch.set_description(f"Epoch {epoch}")

                inputs = data['tokenized_phrases']
                labels = data['one_hot_labels']

                optimizer.zero_grad()
                if USE_CUDA=='cuda':
                    with torch.cuda.amp.autocast():
                        outputs = net(inputs)['output']
                        loss = criterion(outputs, labels)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scheduler.step()
                    scaler.update()
                else:
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                if initial_loss is None:
                    initial_loss = loss.item()
                tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
        #расчет точностей модели после эпохи
        net.eval()
        model_answers=[]
        with tqdm(valloader, unit="batch") as eval_process:
            for data in eval_process:

                inputs = data['tokenized_phrases']

                with torch.no_grad():
                    outputs_dict = net(inputs)
                    pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

                model_answers.append(pred_meddra_code)

        valid_evaluator.get_all_f1_scores(model_answers)
        early_stopping(valid_evaluator.f1_scores_df['All']['f1 macro']['score'], net)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    net.load_state_dict(torch.load(best_model_path))



    print('Evaluating on test dataset')
    net.eval()
    model_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

            model_answers.append(pred_meddra_code)

        test_evaluator.get_all_f1_scores(model_answers)
    all_splits_score_dfs.append(test_evaluator.f1_scores_df)

columns_tuples = [('All', 'f1 micro'), 
                 ('All', 'f1 macro'),

                 ('In VOC', 'f1 micro'), 
                 ('In VOC', 'f1 macro'),

                 ('Out of VOC', 'f1 micro'), 
                 ('Out of VOC', 'f1 macro')]
col_index = pd.MultiIndex.from_tuples(columns_tuples)
f1_mean_scores_df = pd.DataFrame(columns=col_index, index=['mean score', 'std'])
f1_mean_scores_df.loc['std', :] = np.std(np.array(all_splits_score_dfs), axis = 0)
f1_mean_scores_df.loc['mean score', :] = np.mean(np.array(all_splits_score_dfs), axis = 0)
f1_mean_scores_df

Сплит 1


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
100%|██████████| 1503/1503 [00:17<00:00, 85.55batch/s]


All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.695941  0.303515  0.724842  0.357223   0.195122  0.093363

Validation metric increased (-inf --> 0.303515).  Saving model ...


100%|██████████| 1503/1503 [00:17<00:00, 88.26batch/s] 


...

All             In VOC           Out of VOC          
       f1 micro f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.745028  0.31258  0.780781  0.380135   0.118421  0.048009

EarlyStopping counter: 7 out of 7
Early stopping
Evaluating on test dataset


100%|██████████| 2469/2469 [00:25<00:00, 97.17batch/s] 


All              In VOC           Out of VOC          
      f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.73471  0.309099  0.770154  0.402704   0.131387  0.059365

All              In VOC           Out of VOC          
            f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
mean score  0.761452  0.326338  0.797758  0.411703   0.127511  0.071472
std         0.013717  0.013684   0.01462  0.022687   0.015239   0.01367